## Language interpreter

We'll cover the following topics in this notebook:
- Expression interpreter with variables
- Interpreting let-bindings and functions
- Interpreting recursive functions

### Retake: calculator

In [ ]:
type expr = 
	| Float of float
	| Var of string 
	| Neg of expr
	| Add of expr * expr
	| Mult of expr * expr
	(* Extend let bindings *)
	| Let of string * expr * expr

type env = (string * float) list

(* interp : env -> expr -> float *)
let rec interp env = function
	| Float f -> f
	| Var x -> List.assoc x env
	| Neg e -> -1. *. interp env e
	| Add (e1 , e2) -> (interp env e1) +. (interp env e2)
	| Mult (e1, e2) -> (interp env e1) *. (interp env e2)
	(* Let-binding *)
	| Let (str, e1, e2) -> 
		let v = interp env e1 in
			interp ((str, v) :: env) e2

### Functional programming language

In [ ]:

type value = 
	| LitB of bool
	| LitF of float
	| Clo of (value -> value)

type expr = 
	(* Float *)
	| Float of float
	| Var of string 
	| Neg of expr
	| Add of expr * expr
	| Mult of expr * expr
	| Let of string * expr * expr
	(* Functions *)
	| Fun of string * expr
	| App of expr * expr
	(* Booleans *)
	| Bool of bool
	| If of expr * expr * expr
	| And of expr * expr
	| Or of expr * expr
	| Not of expr
	| Eq of expr * expr
	(* Recursion *)
	| RecFun of string * string * expr

exception TypeError

let uop_f op = function
	| LitF f -> LitF (op f)
	| _ -> raise TypeError

let binop_f op v1 v2 = match (v1, v2) with
	| LitF f1 , LitF f2 -> LitF (op f1 f2)
	| _ -> raise TypeError

let rec recur f x = 
	f (recur f) x

let apply v1 v2 = match v1 with
	| Clo f -> f v2
	| _ -> raise TypeError

let uop_b op = function
	| LitB b -> LitB (op b)
	| _ -> raise TypeError

let binop_b op v1 v2 = match (v1, v2) with
	| LitB b1, LitB b2 -> LitB (op b1 b2)
	| _ -> raise TypeError

let eq v1 v2 = match (v1, v2) with
	| LitF f1, LitF f2 -> LitB (f1 = f2)
	| LitB b1, LitB b2 -> LitB (b1 = b2)
	| _ -> raise TypeError

let if_b = function
	| LitB b -> b
	| _ -> raise TypeError

let rec interp env = function
	| Float f -> LitF f
	| Var x -> List.assoc x env
	| Neg e -> uop_f (fun x -> -1. *. x) (interp env e)
	| Add (e1, e2) -> binop_f (+.) (interp env e1) (interp env e2)
	| Mult (e1, e2) -> binop_f ( *. ) (interp env e1) (interp env e2)
	| Let (str, e1, e2) -> 
		let v = interp env e1 in
			interp ((str, v) :: env) e2
	(* Function *)
	| Fun (str, e) -> Clo (fun v -> interp ((str, v) :: env) e)
	| App (e1, e2) -> apply (interp env e1) (interp env e2)
	(* Boolean *)
	| Bool b -> LitB b
	| And (e1, e2) -> binop_b (&&) (interp env e1) (interp env e2)
	| Or (e1, e2) -> binop_b (||) (interp env e1) (interp env e2)
	| Not e -> uop_b not (interp env e)
	| Eq (e1, e2) -> eq (interp env e1) (interp env e2)
	| If (e, e1, e2) -> 
		if if_b (interp env e) then 
			interp env e1
		else 
			interp env e2
	(* Recursive function *)
	| RecFun (name, str, e) -> 
			Clo (recur (fun f v -> interp ((name, Clo f) :: (str, v) :: env) e))

let power = 
	RecFun("power", "x", 
		If( 
			Eq(Var "x", Float 0.),
			Float 1.,
			Mult(Float 2., App(Var "power", Add(Var "x", Float (-1.))))
		)
	)

let test = interp [] (App(power, Float 4.))